# Create predcitors with Prophet and DeepAR+

In [40]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [41]:
# 다른 노트북에서 저장된 변수들을 불러옴
%store -r

In [42]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

## Hyperparameters

In [43]:
forecastHorizon = 13 # 예측할 시간의 갯수 (480시간 == 20일)
NumberOfBacktestWindows = 2 # Backtest를 하는 횟수
BackTestWindowOffset = 13 # Backtest의 평가(Evaluation)의 갯수
ForecastFrequency = "W"

아래 파라미터의 상세 정보는 [여기](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html) 를 보세요.
![ForecastHorizon](static/imgs/ForecastHorizon.png)
![BackTest](static/imgs/BackTest.png)

#### Predictor algorithms

In [44]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet

In [45]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + target_suffix + suffix

In [46]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(
      PredictorName=prophet_predictorName, 
      AlgorithmArn=prophet_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )




### DeepAR+

In [47]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_1' + suffix

In [48]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(
      PredictorName=deeparp_predictorName, 
      AlgorithmArn=deepAR_Plus_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )





## Check if the predictors are active


In [49]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [50]:
#prophet_predictor_arn = 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggleGonsoo_prophet_algo_174655'
target_prophet_predictorArn = prophet_create_predictor_response["PredictorArn"]

In [51]:
target_deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]

In [52]:
%%time
# Check the Prophet status

while True:
    createProphetPredictorStatus = forecast.describe_predictor(PredictorArn= target_prophet_predictorArn)['Status']
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= target_deepar_predictorArn)['Status']    
    print("Prophet: ", createProphetPredictorStatus)
    print("DeepARP: ", createDeeparpPredictorStatus)          
    if createProphetPredictorStatus != 'ACTIVE' and createProphetPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PR

## Predictor evaluation

#### Prophet

In [53]:
# Prophet Metrics
# prophet_arn = prophet_create_predictor_response['PredictorArn']
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=target_prophet_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(prophet_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Prophet',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 93252.788655501,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.03521540844478367,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.06369200343170263,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.029349983106222104,
                                                                                          'Quantile': 0.1}]}},
                                             

In [54]:
# deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]
# forecast.describe_predictor(PredictorArn= deepar_predictorArn)

#### DeepAR+

Same as Prophet and ARIMA, now you should look at the metrics from it.

In [55]:
# DeepAR+ Metrics
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=target_deepar_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 145342.42962620134,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.08020073779742118,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.1057199275735809,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.037648227100967435,
                                                                                          'Quantile': 0.1}]}},
                                      

In [56]:
%store target_prophet_predictorArn
%store target_deepar_predictorArn

Stored 'target_prophet_predictorArn' (str)
Stored 'target_deepar_predictorArn' (str)
